# Keystroke dynamics based biometric authentication

dataset: http://www.cs.cmu.edu/~keystroke/

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
data = pd.read_csv('DSL-StrongPasswordData.csv')

In [3]:
data.head()

,subject,sessionIndex,rep,H.period,DD.period.t,UD.period.t,H.t,DD.t.i,UD.t.i,H.i,...,H.a,DD.a.n,UD.a.n,H.n,DD.n.l,UD.n.l,H.l,DD.l.Return,UD.l.Return,H.Return
0,s002,1,1,0.1491,0.3979,0.2488,0.1069,0.1674,0.0605,0.1169,...,0.1349,0.1484,0.0135,0.0932,0.3515,0.2583,0.1338,0.3509,0.2171,0.0742
1,s002,1,2,0.1111,0.3451,0.2340,0.0694,0.1283,0.0589,0.0908,...,0.1412,0.2558,0.1146,0.1146,0.2642,0.1496,0.0839,0.2756,0.1917,0.0747
2,s002,1,3,0.1328,0.2072,0.0744,0.0731,0.1291,0.0560,0.0821,...,0.1621,0.2332,0.0711,0.1172,0.2705,0.1533,0.1085,0.2847,0.1762,0.0945
3,s002,1,4,0.1291,0.2515,0.1224,0.1059,0.2495,0.1436,0.1040,...,0.1457,0.1629,0.0172,0.0866,0.2341,0.1475,0.0845,0.3232,0.2387,0.0813
4,s002,1,5,0.1249,0.2317,0.1068,0.0895,0.1676,0.0781,0.0903,...,0.1312,0.1582,0.0270,0.0884,0.2517,0.1633,0.0903,0.2517,0.1614,0.0818


In [4]:
data.shape

(20400, 34)

In [5]:
subjects = data["subject"].unique()
subjects

array(['s002', 's003', 's004', 's005', 's007', 's008', 's010', 's011',
       's012', 's013', 's015', 's016', 's017', 's018', 's019', 's020',
       's021', 's022', 's024', 's025', 's026', 's027', 's028', 's029',
       's030', 's031', 's032', 's033', 's034', 's035', 's036', 's037',
       's038', 's039', 's040', 's041', 's042', 's043', 's044', 's046',
       's047', 's048', 's049', 's050', 's051', 's052', 's053', 's054',
       's055', 's056', 's057'], dtype=object)

## Approach

1. Out of 400 samples for each user, 200 used for training 

2. For testing rest 200 will be used along with 5 samples each from the rest 50 users for each user, so a total of 450 samples for testing

3. compare the testing results, both the user's itself and the others for each user and see the difference (intuition: less the difference, more likely) (difference can be in terms of distance or error or anything of that sort)

4. for loops are used, a model is made for each user



### 1. One Class SVM

What if you only have data of one class and the goal is to test new data and found out whether it is alike or not like the training data? A method for this task, which gained much popularity the last two decades, is the One-Class Support Vector Machine. 

It basically separates all the data points from the origin (in feature space F) and maximizes the distance from this hyperplane to the origin. This results in a binary function which captures regions in the input space where the probability density of the data lives. Thus the function returns +1 in a “small” region (capturing the training data points) and −1 elsewhere.

http://rvlasveld.github.io/blog/2013/07/12/introduction-to-one-class-support-vector-machines/

In [6]:
from sklearn import metrics
from sklearn.svm import OneClassSVM

### Equal error rate (ERR)

Equal error rate (EER) is a biometric security system algorithm used to predetermines the threshold values for its false acceptance rate and its false rejection rate. When the rates are equal, the common value is referred to as the equal error rate. The value indicates that the proportion of false acceptances is equal to the proportion of false rejections. The lower the equal error rate value, the higher the accuracy of the biometric system.

FAR(FalseAcceptanceRate) = Numberoffalseacceptance / Numberofidentificationattempt

FRR(FalseRejectionRate) = Numberoffalserejection / Numberofidentificationattempt

Now, plot FAR and FRR on a graph and you will get a point where two curves generated by FAR and FRR intersects each other. That point is the EER. That means, EER is the point where your false identification and false rejection rate are minimal and optimal. The lower your EER, the better your system.

ACC = 100 - ERR

In [7]:
def evaluateEER(user_scores, imposter_scores):
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    fpr, tpr, thresholds = metrics.roc_curve(labels, user_scores + imposter_scores)
    frrates = 1 - tpr
    farates = fpr
    dists = frrates - farates
    idx1 = np.argmin(dists[dists >= 0])
    idx2 = np.argmax(dists[dists < 0])
    x = [frrates[idx1], farates[idx1]]
    y = [frrates[idx2], farates[idx2]]
    a = ( x[0] - x[1] ) / ( y[1] - x[1] - y[0] + x[0] )
    eer = x[0] + a * ( y[0] - x[0] )
    acc = 1 - eer
    return eer, acc

In [8]:
# preparing training and testing data for each user
eers = []
accs = []
for subj in subjects:
    user_data = data.loc[data.subject == subj, 'H.period':'H.Return']
    train_data = user_data[:200]
    user_test_data = user_data[200:]
    imposter_test_data = data.loc[data.subject != subj,:]
    imposter_test_data = imposter_test_data.groupby('subject').head(5).loc[:, "H.period":"H.Return"]
    test_data = user_test_data.append(imposter_test_data)
    clf = OneClassSVM(kernel='linear',gamma=0.001, nu=0.95)
    clf.fit(train_data)
    u_scores = clf.decision_function(user_test_data)
    i_scores = clf.decision_function(imposter_test_data)
    u_scores = list(u_scores)
    i_scores = list(i_scores)
    fraud_pred = clf.predict(test_data)
    eer, acc = evaluateEER(u_scores, i_scores)
    eers.append(eer)
    accs.append(acc)
print(np.mean(eers))
print(np.mean(accs))


0.175627732540059
0.8243722674599411


### 2. Neural Network

In [6]:
from sklearn.model_selection import train_test_split

In [14]:
x = data.loc[data.subject == 's002', 'H.period':'H.Return']
y = data.loc[data.subject == 's002', 'subject']
for subj in subjects[1:]:
    x = x.append(data.loc[data.subject == subj, 'H.period':'H.Return'])
    y = y.append(data.loc[data.subject == subj, 'subject'])

In [15]:
from sklearn import preprocessing

In [16]:
y = pd.DataFrame(y)
y.shape

(20400, 1)

In [17]:
le = preprocessing.LabelEncoder()
Y = y.apply(le.fit_transform)

In [18]:
enc = preprocessing.OneHotEncoder()
enc.fit(Y)
onehotlabels = enc.transform(Y).toarray()
onehotlabels.shape

C:\Users\Karthika\Anaconda2\envs\jupyter36\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(20400, 51)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(x, onehotlabels, test_size=0.3, random_state=0, stratify=onehotlabels)

In [20]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(14280, 31)
(14280, 51)
(6120, 31)
(6120, 51)


In [36]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation

network = Sequential()
network.add(Dense(25,input_dim=31,activation='relu'))
network.add(Dense(25,activation='relu'))
network.add(Dense(51, activation = 'softmax'))
network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 25)                800       
_________________________________________________________________
dense_11 (Dense)             (None, 25)                650       
_________________________________________________________________
dense_12 (Dense)             (None, 51)                1326      
Total params: 2,776
Trainable params: 2,776
Non-trainable params: 0
_________________________________________________________________


In [37]:
network.compile(loss='categorical_crossentropy',optimizer='adam', metrics = ['accuracy'])
network.fit(X_train,y_train,epochs=300,batch_size=100,validation_split=0.2)

Train on 11424 samples, validate on 2856 samples
Epoch 1/2000
11424/11424 [==============================] - 2s 140us/step - loss: 3.9013 - acc: 0.0460 - val_loss: 3.8313 - val_acc: 0.0690
Epoch 2/2000
11424/11424 [==============================] - 1s 65us/step - loss: 3.6761 - acc: 0.0693 - val_loss: 3.4970 - val_acc: 0.0732
Epoch 3/2000
11424/11424 [==============================] - 1s 70us/step - loss: 3.3793 - acc: 0.1181 - val_loss: 3.2385 - val_acc: 0.1632
Epoch 4/2000
11424/11424 [==============================] - 1s 70us/step - loss: 3.1606 - acc: 0.1866 - val_loss: 3.0524 - val_acc: 0.2167
Epoch 5/2000
11424/11424 [==============================] - 1s 67us/step - loss: 3.0011 - acc: 0.2540 - val_loss: 2.9111 - val_acc: 0.2850
Epoch 6/2000
11424/11424 [==============================] - 1s 67us/step - loss: 2.8653 - acc: 0.3122 - val_loss: 2.7875 - val_acc: 0.3442
Epoch 7/2000
11424/11424 [==============================] - 1s 89us/step - loss: 2.7344 - acc: 0.3510 - val_loss: 2.

In [38]:
test_scores = network.evaluate(X_test, y_test, verbose = 0)
print('Test accuracy:', test_scores[1])

Test accuracy: 0.888235294117647


In [39]:
y_pred = network.predict(X_test[22:23])
y_pred.argmax()

48

In [40]:
y_test[22:23].argmax()

48